In [ ]:
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import models, layers
from sklearn.model_selection import train_test_split

In [ ]:
meta = pd.read_csv('../processed_data/train.csv', index_col=0)
meta.head()

### Metadata

In [ ]:
y = meta['target']
X = meta.drop('target', axis=1)
meta_train, meta_test, y_train, y_test = train_test_split(X, y, test_size=0.3 ,random_state=867_5309)

In [ ]:
meta_train = meta_train.astype('ushort')
meta_test = meta_test.astype('ushort')

In [ ]:
meta_train.shape

In [ ]:
y_train.shape

### Image data

In [ ]:
TRAIN_PATH = '../processed_data/train_img_array.pkl'
TEST_PATH = '../processed_data/test_img_array.pkl'
with open(TRAIN_PATH, 'rb') as file:
    img_train = pickle.load(file)
with open(TEST_PATH, 'rb') as file:
    img_test = pickle.load(file)
img_train = img_train.astype('ushort')
img_test = img_test.astype('ushort')

In [ ]:
img_train.shape

In [ ]:
img_test.shape

### Model

This model with utilize the functional Keras API to produce a mixed model.

In [ ]:
# Define inputs
meta_inputs = tf.keras.Input(shape=(2065,))
img_inputs = tf.keras.Input(shape=(80,120,3,))

# Model 1
meta_layer1 = tf.keras.layers.Dense(64, activation='relu')(meta_inputs)
meta_output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(meta_layer1)

# Model 2
img_conv_layer1 = tf.keras.layers.Conv2D(32, kernel_size=(3,3), padding='same', activation='relu')(img_inputs)
img_pooling_layer1 = tf.keras.layers.MaxPooling2D()(img_conv_layer1)
img_conv_layer2 = tf.keras.layers.Conv2D(64, kernel_size=(3,3), padding='same', activation='relu')(img_pooling_layer1)
img_pooling_layer2 = tf.keras.layers.MaxPooling2D()(img_conv_layer2)
img_flatten_layer = tf.keras.layers.Flatten()(img_pooling_layer2)
img_dense_layer = tf.keras.layers.Dense(1024, activation='relu')(img_flatten_layer)
img_output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(img_dense_layer)

# Merge models
merged = tf.keras.layers.add([meta_output_layer, img_output_layer])

# Define functional model
model = tf.keras.Model(inputs=[meta_inputs, img_inputs], outputs=merged)

# Compile model
auc = tf.keras.metrics.AUC(name = 'auc')
model.compile('adam', loss='binary_crossentropy', metrics=[auc])
model.summary()

In [ ]:
epochs = 15
history = model.fit(meta_X_train, meta_y_train, epochs=epochs, batch_size=15_000, validation_data=(X_test, y_test))

In [ ]:
X = np.arange(0, epochs)
plt.figure(1, figsize = (20, 12))
plt.subplot(1,2,1)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(X, history.history['loss'], label = 'Training Loss')
plt.plot(X, history.history['val_loss'], label = 'Validation Loss')
plt.grid(True)
plt.xticks(X)
plt.legend()
plt.title('Loss', fontsize=30)

plt.subplot(1,2,2)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.plot(X, history.history['auc'], label = 'Training AUC')
plt.plot(X, history.history['val_auc'], label = 'Validation AUC')
plt.grid(True)
plt.xticks(X)
plt.legend()
plt.title('AUC', fontsize=30)